In [141]:
import torch
import requests
import os
import torch.nn as nn
import torchvision
from torch.utils.data import DataLoader
from torchvision import datasets, transforms
import random
from PIL import Image
import numpy as np
import matplotlib.pyplot as plt
from pathlib import Path
import zipfile
from tqdm.auto import tqdm
from typing import Dict, List, Tuple
import cv2
combat = "combat"
rehab = "humanitarianaid"
military_vehicles = "militaryvehicles"
fire = "fire"
destroyed_building = "destroyedbuilding"
device = 'cuda'

In [142]:
def event_identification(img):        # NOTE: You can tweak this function in case you need to give more inputs 
    event_list = []
    im5 = img[879:963,197:281] #A
    im4 = img[678:762,670:754] #B
    im2 = img[470:558,678:763] #C
    im3 = img[465:552,179:266] #D
    im1 = img[128:216,200:285] #E

    event_list.append(im5)
    event_list.append(im4)
    event_list.append(im2)
    event_list.append(im3)
    event_list.append(im1)
    # event_list.reverse()
    cv2.imwrite("s1.jpg", im1)
    cv2.imwrite("s2.jpg", im2)
    cv2.imwrite("s3.jpg", im3)
    cv2.imwrite("s4.jpg", im4)
    cv2.imwrite("s5.jpg", im5)
    return event_list

In [143]:
def classify_event(image):
    cv2.imwrite("a.jpg",image)
    from PIL import Image
    import torchvision.transforms as transforms
    import torch # Use the input size corresponding to the model
    test_image = Image.open("a.jpg")
    manual_transforms = transforms.Compose([
    transforms.Resize((150, 150), antialias=True),
    # transforms.Resize((244, 244), antialias=True),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])
    
    
    test_image = manual_transforms(test_image)
    model_path = '/home/adi/GG_1267/Task_1A_git/Task_4a/modelx.pth'  #model uploaded on google drive
    model = torch.load(model_path,map_location='cpu')
    model.eval()

    with torch.no_grad():
        output = model(test_image.unsqueeze(0))
    target_image_pred = output
    target_image_pred_probs = torch.softmax(target_image_pred, dim=1)
    target_image_pred_label = torch.argmax(target_image_pred_probs, dim=1)
    class_names = ["combat", "destroyedbuilding", "fire", "humanitarianaid", "militaryvehicles"]
    # os.remove("a.png")
    return class_names[target_image_pred_label.item()]

In [144]:
def classification(event_list):
    detected_list = []
    for img_index in range(0,5):
        img = event_list[img_index]
        detected_event = classify_event(img)
        print((img_index + 1), detected_event)
        if detected_event == combat:
            detected_list.append("combat")
        if detected_event == rehab:
            detected_list.append("rehab")
        if detected_event == military_vehicles:
            detected_list.append("militaryvehicles")
        if detected_event == fire:
            detected_list.append("fire")
        if detected_event == destroyed_building:
            detected_list.append("destroyedbuilding")
    return detected_list

In [145]:
imager = cv2.imread("/home/adi/GG_1267/Task_1A_git/Task_4a/eyantra photos/saved_image24.jpg")
event_list = event_identification(imager)
print(classification(event_list))

1 combat
2 fire
3 destroyedbuilding
4 militaryvehicles
5 humanitarianaid
['combat', 'fire', 'destroyedbuilding', 'militaryvehicles', 'rehab']
